In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os 
import random
from random import shuffle 
from PIL import Image
from tqdm import tqdm 
from PIL import Image

from imageio import imread
import scipy.ndimage
import scipy 
import seaborn as sns

from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import os
# print(os.listdir("C:/Users/vasud/Desktop/MS in Software Engineering SJSU/Fall 2020/255 - Data Mining/Rentinal Classification Project"))
path = "../input/diabetic-retinopathy-2015-data-colored-resized/colored_images/"

In [ ]:
labels = pd.read_csv('../input/diabetic-retinopathy-2015-data-colored-resized/trainLabels.csv')
labels.head()

In [ ]:
labels["level"].value_counts().plot(kind="bar")

In [ ]:
import os
from glob import glob
result = [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.png'))]

In [ ]:
# %%time
# # a = [ cv2.imread(x) for x in result if '/'+level_0[0] in x] [0]
# level_0 = labels["image"][labels['level'] == 0].tolist()
# a =[]
# for row in level_0:
#     a.append([cv2.imread(x) for x in result if '/'+row in x][0])

In [ ]:
# %%time
# c = [cv2.imread(x) for row in level_0 for x in result if '/'+row in x]

In [ ]:
%%time
level_0 = labels["image"][labels['level'] == 0].tolist()
level_1 = labels["image"][labels['level'] == 1].tolist()
level_2 = labels["image"][labels['level'] == 2].tolist()
level_3 = labels["image"][labels['level'] == 3].tolist()
level_4 = labels["image"][labels['level'] == 4].tolist()

no_DR = random.choices(level_0, k=2832)
level_1 = random.choices(level_1,k=708)
level_2 = random.choices(level_2,k=708)
level_3 = random.choices(level_3,k=708)
level_4 = random.choices(level_4,k=708)
with_DR = np.concatenate((level_1,level_2,level_3,level_4))

# Train_no_DR = [cv2.imread(path+"/"+str(x)+".jpeg") for x in random.choices(no_DR, k=2832)]
# Train_with_DR = [cv2.imread(path+"/"+str(x)+".jpeg") for x in with_DR]
Train_no_DR = [cv2.imread(x) for row in no_DR for x in result if '/'+row in x]
print("Train_no_DR done")
Train_with_DR = [cv2.imread(x) for row in with_DR for x in result if '/'+row in x]
image_size = 128


# Y Train values
y_Train_no_DR = [0]*len(no_DR)
y_Train_with_DR = [1]*len(level_1) + [2]*len(level_2) + [3]*len(level_3) + [4]*len(level_4)

In [ ]:
len(y_Train_no_DR),len(y_Train_with_DR)

In [ ]:
# level_0 = labels["image"][labels['level'] == 0].tolist()
# level_1 = labels["image"][labels['level'] == 1].tolist()
# level_2 = labels["image"][labels['level'] == 2].tolist()
# level_3 = labels["image"][labels['level'] == 3].tolist()
# level_4 = labels["image"][labels['level'] == 4].tolist()

# no_DR = level_0
# with_DR = np.concatenate((level_1,level_2,level_3,level_4))

# Train_no_DR = [cv2.imread(path+"/"+str(x)+".jpeg") for x in random.choices(no_DR, k=2832)]
# Train_with_DR = [cv2.imread(path+"/"+str(x)+".jpeg") for x in random.choices(with_DR, k=2832)]
# image_size = 128

In [ ]:
# Test_no_DR = "C:/Users/vasud/Desktop/MS in Software Engineering SJSU/Fall 2020/255 - Data Mining/Rentinal Classification Project/resized_train/resized_test/DRIVE/test/images"
# Test_with_DR = "C:/Users/vasud/Desktop/MS in Software Engineering SJSU/Fall 2020/255 - Data Mining/Rentinal Classification Project/resized_train/resized_test/DRIVE/training/images"

In [ ]:
# len(Test_with_DR)

In [ ]:
Image.fromarray(Train_no_DR[2083])

In [ ]:
Image.fromarray(Train_with_DR[0])

In [ ]:
exp_lis = []
for image in Train_no_DR:
    try:
        img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (image_size, image_size)).flatten()
        np_img = np.asarray(img)
    except:
        exp_lis.append(image)
    
for image2 in Train_with_DR:
    try:
        img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        img2 = cv2.resize(img2, (image_size, image_size)).flatten()
        np_img2 = np.asarray(img2)
    except:
        exp_lis.append(image2)

plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.imshow(np_img.reshape(image_size, image_size))
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(np_img2.reshape(image_size, image_size))
plt.axis('off')
plt.title("images with and without Diabetic Retenopathy in GrayScale")

In [ ]:
print(exp_lis)

In [ ]:
def train_data():
    train_data_no_DR = [] 
    train_data_with_DR=[]
    exp_list = []
    for image1 in Train_no_DR:
        try:
            img1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
            img1 = cv2.resize(img1, (image_size, image_size))
            train_data_no_DR.append(img1)
        except:
            exp_list.append(image)
        
    for image2 in Train_with_DR:
        try:
            img2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
            img2 = cv2.resize(img2, (image_size, image_size))
            train_data_with_DR.append(img2)
        except:
            exp_list.append(image2)
 
    train_data= np.concatenate((np.asarray(train_data_no_DR),np.asarray(train_data_with_DR)),axis=0)
    return train_data 

In [ ]:
def test_data():
    test_data_no_DR = [] 
    test_data_with_DR=[]
    for image1 in tqdm(os.listdir(Test_no_DR)): 
        path = os.path.join(Test_no_DR, image1)
        img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img1 = cv2.resize(img1, (image_size, image_size))
        test_data_no_DR.append(img1)
        
    for image2 in tqdm(os.listdir(Test_with_DR)): 
        path = os.path.join(Test_with_DR, image2)
        img2 = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
        img2 = cv2.resize(img2, (image_size, image_size))
        test_data_with_DR.append(img2) 
    
    test_data= np.concatenate((np.asarray(test_data_no_DR),np.asarray(test_data_with_DR)),axis=0) 
    return test_data 


In [ ]:
traindata = train_data() 
# testdata = test_data()

In [ ]:
# print(len(testdata),len(traindata))

In [ ]:
x_data = traindata
x_data = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [ ]:
# x_data = np.concatenate((traindata,testdata),axis=0)
# x_data = (x_data-np.min(x_data))/(np.max(x_data)-np.min(x_data))

In [ ]:
# z1 = np.zeros(2832)
# o1 = np.ones(2832)
# Y_train = np.concatenate((o1, z1), axis=0)
# z = np.zeros(20)
# o = np.ones(20)
# Y_test = np.concatenate((o, z), axis=0)

In [ ]:
# y_data = np.concatenate((Y_train,Y_test),axis=0).reshape(x_data.shape[0],1)

In [ ]:
y_data = pd.Series(y_Train_no_DR+y_Train_with_DR)

In [ ]:
y_data.value_counts()

In [ ]:
print("X shape: " , x_data.shape)
print("Y shape: " , len(y_data))

In [ ]:
number_of_data = x_data.shape[0]
x_flatten = x_data.reshape(number_of_data,x_data.shape[1]*x_data.shape[2])
print("X flatten",x_flatten.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_flatten, y_data, test_size=0.15, random_state=42)


In [ ]:
print("x train: ",x_train.shape)
print("x test: ",x_test.shape)
print("y train: ",y_train.shape)
print("y test: ",y_test.shape)

In [ ]:
%%time
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

In [ ]:
y_pred = logreg.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
print(classification_report(y_train, logreg.predict(x_train)))

In [ ]:
logreg.score(x_train,y_train),\
logreg.score(x_test,y_test)